In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, transforms, models

# from torchvision.models import resnet18, ResNet18_Weights
from torchvision.models import resnext101_64x4d, ResNeXt101_64X4D_Weights

from torch.utils.data import DataLoader
import torch.backends.cudnn as cudnn

from PIL import Image

import os, time, copy

import os
import random
import matplotlib.pyplot as plt
from PIL import Image
import torchvision.transforms as transforms
from torchvision.transforms import v2

cudnn.benchmark = True
plt.ion()

In [2]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")
num_classes = 10
num_epochs = 20
batch_size = 128
learning_rate = 0.001

Using cuda device


# **Xử lí data**

In [3]:
data_dir = "/kaggle/input/handwritten-tl/digits_data"
train_dir = os.path.join(data_dir, "train")
val_dir = os.path.join(data_dir, "val")

In [4]:
class InvertIfNeeded:
    def __call__(self, img):
        tensor = transforms.functional.to_tensor(img)
        if tensor.mean() < 0.3:
            tensor = 1.0 - tensor
        return transforms.functional.to_pil_image(tensor)

# data_transforms = {
#     'train': transforms.Compose([
#         transforms.Grayscale(3),  #ResNet cần 3 kênh
#         transforms.Resize((224, 224)),
#         transforms.RandomHorizontalFlip(),
#         transforms.ToTensor(),
#         transforms.Normalize([0.5]*3, [0.5]*3),
#     ]),
#     'val': transforms.Compose([
#         transforms.Grayscale(3),
#         transforms.Resize((224, 224)),
#         transforms.ToTensor(),
#         transforms.Normalize([0.5]*3, [0.5]*3),
#     ]),
# }

# data_transforms = {
#     'train': transforms.Compose([
#         transforms.Grayscale(3),
#         transforms.Resize((224, 224)),
#         transforms.RandomRotation(15),
#         transforms.RandomAffine(degrees=0, translate=(0.05, 0.05)),
#         transforms.RandomHorizontalFlip(),
#         transforms.ToTensor(),
#         transforms.Normalize([0.5]*3, [0.5]*3),
#     ]),
#     'val': transforms.Compose([
#         transforms.Grayscale(3),
#         transforms.Resize((224, 224)),
#         transforms.ToTensor(),
#         transforms.Normalize([0.5]*3, [0.5]*3),
#     ]),
# }

mean = [0.485, 0.456, 0.406]
std  = [0.229, 0.224, 0.225]

br = 0.65
ct = 0.6

data_transforms = {
    'train': v2.Compose([
        v2.ToImage(),  # đảm bảo đầu vào là ảnh
        v2.Resize((224, 224)),
        v2.ColorJitter(brightness=(br, br), contrast=(ct, ct)),
        v2.GaussianBlur(kernel_size=(5, 5), sigma=2.5),
        v2.RandomRotation(15),
        v2.RandomAffine(degrees=0, translate=(0.05, 0.05)),
        v2.RandomHorizontalFlip(),
        v2.ToDtype(torch.float32, scale=True),
        v2.Normalize(mean=mean, std=std),
    ]),
    'val': v2.Compose([
        v2.ToImage(),
        v2.Resize((224, 224)),
        v2.ColorJitter(brightness=(br, br), contrast=(ct, ct)),
        v2.GaussianBlur(kernel_size=(5, 5), sigma=2.5),
        v2.ToDtype(torch.float32, scale=True),
        v2.Normalize(mean=mean, std=std),
    ]),
}

In [5]:
def remove_corrupted_images(folder):
    for root, _, files in os.walk(folder):
        for fname in files:
            fpath = os.path.join(root, fname)
            try:
                img = Image.open(fpath)
                img.verify()  # kiểm tra định dạng ảnh
            except Exception as e:
                print(f"🗑️ Xóa ảnh lỗi: {fpath}")
                os.remove(fpath)

remove_corrupted_images(train_dir)
remove_corrupted_images(val_dir)

In [6]:
image_datasets = {
    'train': datasets.ImageFolder(train_dir, transform=data_transforms['train']),
    'val': datasets.ImageFolder(val_dir, transform=data_transforms['val']),
}
dataloaders = {
    x: DataLoader(image_datasets[x], batch_size=batch_size, shuffle=(x=='train'))
    for x in ['train', 'val']
}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
dataset_sizes

{'train': 5762, 'val': 1310}

# **Transfer learning với Resnet**

In [7]:
# model = resnet18(weights='IMAGENET1K_V1')

weights = ResNeXt101_64X4D_Weights.IMAGENET1K_V1
model = resnext101_64x4d(weights=weights)

for param in model.parameters():
    param.requires_grad = False  # Freeze toàn bộ layers

# Thay lớp fully-connected để phù hợp với 10 lớp
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)


criterion = nn.CrossEntropyLoss()

# Optim 
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)


#Optim SGD
# optimizer = optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
# exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

Downloading: "https://download.pytorch.org/models/resnext101_64x4d-173b62eb.pth" to /root/.cache/torch/hub/checkpoints/resnext101_64x4d-173b62eb.pth
100%|██████████| 319M/319M [00:13<00:00, 24.6MB/s]


In [8]:
import sys
import time
import copy
from tqdm import tqdm
from contextlib import redirect_stdout

# Logging class
class DualWriter:
    def __init__(self, file):
        self.file = file
        self.stdout = sys.__stdout__

    def write(self, text):
        self.stdout.write(text)
        self.file.write(text)

    def flush(self):
        self.stdout.flush()
        self.file.flush()

In [9]:
from tqdm import tqdm

def train_model(model, criterion, optimizer, num_epochs, log_path):
    log_file = open(log_path, "a")
    dual_output = DualWriter(log_file)

    with redirect_stdout(dual_output):
        best_model_wts = copy.deepcopy(model.state_dict())
        best_acc = 0.0
        since = time.time()

        for epoch in range(num_epochs):
            print(f'\nEpoch {epoch+1}/{num_epochs}')
            for phase in ['train', 'val']:
                if phase == 'train':
                    model.train()
                else:
                    model.eval()

                running_loss = 0.0
                running_corrects = 0

                for inputs, labels in tqdm(dataloaders[phase], desc=f"{phase} Epoch {epoch+1}"):
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    optimizer.zero_grad()
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)
                        preds = torch.argmax(outputs, 1)

                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels).item()

                epoch_loss = running_loss / dataset_sizes[phase]
                epoch_acc = running_corrects / dataset_sizes[phase]
                print(f'{phase.capitalize()} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

                if phase == 'val' and epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model_wts = copy.deepcopy(model.state_dict())

        time_elapsed = time.time() - since
        print(f'\nTraining complete in {time_elapsed//60:.0f}m {time_elapsed%60:.0f}s')
        print(f'Best val Acc: {best_acc:.4f}')

        model.load_state_dict(best_model_wts)

    print("🟢 Đã thoát khỏi logging context. Giờ chỉ in ra console.")
    log_file.close()
    return model, best_acc, time_elapsed

In [10]:
# best_model, best_acc, time_elapsed = train_model(model, criterion, optimizer,  
#                                                  exp_lr_scheduler, num_epochs)
log_path = 'train_log.txt'
best_model, best_acc, time_elapsed = train_model(model, criterion, optimizer, num_epochs, log_path)

train Epoch 1:   0%|          | 0/46 [00:00<?, ?it/s]


Epoch 1/20


val Epoch 1:   0%|          | 0/11 [00:00<?, ?it/s]

Train Loss: 2.2067 Acc: 0.1947


train Epoch 2:   0%|          | 0/46 [00:00<?, ?it/s]

...

Val Loss: 1.4086 Acc: 0.5542

Epoch 20/20


val Epoch 20:   0%|          | 0/11 [00:00<?, ?it/s]

Train Loss: 1.3562 Acc: 0.5961


val Epoch 20: 100%|██████████| 11/11 [01:18<00:00,  7.12s/it]

🟢 Đã thoát khỏi logging context. Giờ chỉ in ra console.


# **Lưu:**
- Model
- Best accuracy
- Thời gian chạy
- Optimizer
- Hàm loss
- Transform data

In [11]:
torch.save(model.state_dict(), 'resnext101_64x4d.pth')

log_path = "final_train_log.txt"
with open(log_path, "a") as f:
    f.write("="*60 + "\n")
    f.write("Training Summary\n")
    f.write("="*60 + "\n")
    f.write(f"Time: {time_elapsed//60:.0f}m {time_elapsed%60:.0f}s\n")
    f.write(f"Best Validation Accuracy: {best_acc:.4f}\n")
    f.write(f"\nOptimizer: {optimizer.__class__.__name__} - {optimizer.state_dict()['param_groups'][0]}\n")
    f.write(f"Loss Function: {criterion.__class__.__name__}\n")

    f.write("\nTransforms:\n")
    for phase in ['train', 'val']:
        f.write(f"  {phase}:\n")
        for t in data_transforms[phase].transforms:
            f.write(f"    - {t}\n")

    f.write("="*60 + "\n\n")

# **Test 20 ảnh random trên tập test**

In [12]:
# # Định nghĩa thư mục test
# test_folder = "/kaggle/input/handwritten-test-10k"

# # Thiết bị (CPU/GPU)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Định nghĩa transform cho test (tương tự 'val')
# transform = transforms.Compose([
#     InvertIfNeeded(),
#     transforms.Grayscale(3),  # Chuyển thành 3 kênh cho ResNet
#     transforms.Resize((224, 224)),  # Thay đổi kích thước ảnh
#     transforms.ToTensor(),  # Chuyển thành tensor
#     transforms.Normalize([0.5]*3, [0.5]*3),  # Chuẩn hóa
# ])

# # Lấy 20 ảnh ngẫu nhiên từ thư mục test
# all_files = [f for f in os.listdir(test_folder)]
# random.shuffle(all_files)
# selected_files = all_files[:20]

# # Chuyển mô hình sang chế độ đánh giá
# model.eval()

# # Tạo figure để plot
# plt.figure(figsize=(12, 6))

# # Test và plot từng ảnh
# for i, filename in enumerate(selected_files):
#     # Đọc ảnh
#     img_path = os.path.join(test_folder, filename)
#     image = Image.open(img_path).convert("RGB")  # Đọc ảnh và chuyển thành grayscale

#     # Áp dụng transform
#     image_input = transform(image).unsqueeze(0).to(device)  # Thêm chiều batch và chuyển sang device

#     # Dự đoán
#     with torch.no_grad():
#         outputs = model(image_input)
#         _, predicted = torch.max(outputs, 1)
#         pred_label = predicted.item()

#     # Plot ảnh
#     plt.subplot(4, 5, i + 1)
#     plt.imshow(image, cmap="gray")  # Hiển thị ảnh gốc (grayscale)
#     plt.title(f"Pred: {pred_label}")
#     plt.axis("off")

# # Điều chỉnh layout và hiển thị
# plt.tight_layout()
# plt.show()

# **Dự đoán cho dataset 2k**

In [13]:
with open("/kaggle/input/cs114-heic-name-10ktest/heic_name.txt", "r", encoding="utf-8") as f:
    heic_names = [line.strip() for line in f]

# print(heic_names)

In [ ]:
# Định nghĩa thư mục test và file output
test_folder = "/kaggle/input/handwritten-test-2k"
output_file = "predict_2k.txt"

# Thiết bị (CPU/GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Định nghĩa transform cho test (tương tự 'val')
# transform = transforms.Compose([
#     InvertIfNeeded(),
#     transforms.Grayscale(3),  # Chuyển thành 3 kênh cho ResNet
#     transforms.Resize((224, 224)),  # Thay đổi kích thước ảnh
#     transforms.ToTensor(),  # Chuyển thành tensor
#     transforms.Normalize([0.5]*3, [0.5]*3),  # Chuẩn hóa
# ])
transform = v2.Compose([
        v2.ToImage(),
        v2.Resize((224, 224)),
        v2.ColorJitter(brightness=(br, br), contrast=(ct, ct)),
        v2.GaussianBlur(kernel_size=(5, 5), sigma=2.5),
        v2.ToDtype(torch.float32, scale=True),
        v2.Normalize(mean=mean, std=std),
    ])

# Lấy tất cả ảnh trong thư mục test
all_files = [f for f in os.listdir(test_folder)]

count = 0
# Chuyển mô hình sang chế độ đánh giá
model.eval()

# Mở file để lưu kết quả
with open(output_file, "w") as f:
    # Test từng ảnh
    for filename in all_files:
        # Đọc ảnh
        part = os.path.splitext(filename)
        name = part[0]
        ext = part[1].lower()
        # ext = os.path.splitext(filename)[-1].lower()
        
        # if ext == ".heic" or ext == ".md" :
        #     print("Lỗi heic, .md : ", filename)
        #     continue
            
        img_path = os.path.join(test_folder, filename)
        image = Image.open(img_path).convert("RGB")  # Đọc ảnh và chuyển thành grayscale

        # Áp dụng transform
        image_input = transform(image).unsqueeze(0).to(device)  # Thêm chiều batch và chuyển sang device

        # Dự đoán
        with torch.no_grad():
            outputs = model(image_input)
            _, predicted = torch.max(outputs, 1)
            pred_label = predicted.item()
            
        if name in heic_names:
            filename = name + '.HEIC'
            print(filename)
        # Ghi vào file với định dạng img_name,label
        f.write(f"{filename},{pred_label}\n")
        count += 1

print('Tổng số test sample : ', count)

0e308e4279faba22843c007c709d047f  -.HEIC
bfa13efb7012f26ec808491f1b43d8eb  -.HEIC
240c99ed2f4fc115292ee39e630307d1  -.HEIC
d4b5381a27ee385fcb5e5f1896edcc68  -.HEIC
4125170c9b33affe28532d84097fef4a  -.HEIC
d67a99873641d73fffd0498ce135dec9  -.HEIC
ee5f335ad27f7817568c36721924c8e9  -.HEIC
dbd025557c0ff34f9099de9d421d8ebb  -.HEIC
faff9da17885d6589a9775ccdecb8b7d  -.HEIC
2cc4b987bfe65eb3beda9870d3dd5571  -.HEIC
be633c756666b255f89a23ae798d690d  -.HEIC
bc137c1dc2466da50b14cc39a540ba64  -.HEIC
379e725180d1877eee670939084c0361  -.HEIC
346c9d49d4f66bd42b90d9f034f240f6  -.HEIC
02c41345f52a4c87bcf29c0aed37a9ca  -.HEIC
79cc598c7c0fa1380a771bf724668662  -.HEIC
4056e13b8d7ffb1d3f69a64d3d742919  -.HEIC
8627553fc3ae84da85df6ee275ba7a3c  -.HEIC
9d2cf96b868cc57eafeb7130fcd9edb4  -.HEIC
a551c30daf44700208d8ed24d70db855  -.HEIC
c6ce9f623fc4dd820f40fad5f541c87e  -.HEIC
8e4a38bd19c94970f342494bb01213d8  -.HEIC
18b3189fa0aecab82ae817e8155a9844  -.HEIC
f335ebd8f31b916c71ddd182bf2f2e40  -.HEIC
3edc952b8034e0d9

# **Dự đoán cho dataset 10k**

In [ ]:
# Định nghĩa thư mục test và file output
test_folder = "/kaggle/input/handwritten-test-10k"
output_file = "predict_10k.txt"

# Thiết bị (CPU/GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Định nghĩa transform cho test (tương tự 'val')
# transform = transforms.Compose([
#     InvertIfNeeded(),
#     transforms.Grayscale(3),  # Chuyển thành 3 kênh cho ResNet
#     transforms.Resize((224, 224)),  # Thay đổi kích thước ảnh
#     transforms.ToTensor(),  # Chuyển thành tensor
#     transforms.Normalize([0.5]*3, [0.5]*3),  # Chuẩn hóa
# ])
transform = v2.Compose([
        v2.ToImage(),
        v2.Resize((224, 224)),
        v2.ColorJitter(brightness=(br, br), contrast=(ct, ct)),
        v2.GaussianBlur(kernel_size=(5, 5), sigma=2.5),
        v2.ToDtype(torch.float32, scale=True),
        v2.Normalize(mean=mean, std=std),
    ])

# Lấy tất cả ảnh trong thư mục test
all_files = [f for f in os.listdir(test_folder)]

count = 0
# Chuyển mô hình sang chế độ đánh giá
model.eval()

# Mở file để lưu kết quả
with open(output_file, "w") as f:
    # Test từng ảnh
    for filename in all_files:
        # Đọc ảnh
        part = os.path.splitext(filename)
        name = part[0]
        ext = part[1].lower()
        # ext = os.path.splitext(filename)[-1].lower()
        
        # if ext == ".heic" or ext == ".md" :
        #     print("Lỗi heic, .md : ", filename)
        #     continue
            
        img_path = os.path.join(test_folder, filename)
        image = Image.open(img_path).convert("RGB")  # Đọc ảnh và chuyển thành grayscale

        # Áp dụng transform
        image_input = transform(image).unsqueeze(0).to(device)  # Thêm chiều batch và chuyển sang device

        # Dự đoán
        with torch.no_grad():
            outputs = model(image_input)
            _, predicted = torch.max(outputs, 1)
            pred_label = predicted.item()
            
        if name in heic_names:
            filename = name + '.HEIC'
            print(filename)
        # Ghi vào file với định dạng img_name,label
        f.write(f"{filename},{pred_label}\n")
        count += 1

print('Tổng số test sample : ', count)

2e5ab66c892d882503a710aa97b7e15b.HEIC
f986a68ead56729a5fd52fd338cc35a6.HEIC
0e308e4279faba22843c007c709d047f  -.HEIC
614a8c12dd3771cedcabcdb3642ba95b.HEIC
9e1f3378e66e8cd4c1f795d8e642fc60.HEIC
064e9f0be1e982a8f64ad6410df288c7.HEIC
b97120946046fd8a6febca02f8b9ac5b.HEIC
f36e0a9c5b7bab3d8a83c61f60144bf6.HEIC
2a96d698456e7c0918d136f7e3372847.HEIC
bfa13efb7012f26ec808491f1b43d8eb  -.HEIC
240c99ed2f4fc115292ee39e630307d1  -.HEIC
d4b5381a27ee385fcb5e5f1896edcc68  -.HEIC
4246ea371a0a9096bb9b3d9048b7402d.HEIC
9ff224036fcd320556163e4b2feb813a.HEIC
313ec17e81da537717027fd5cb2e7d76.HEIC
7b144c8aee2c11ad0a54433ccadb4a66.HEIC
4125170c9b33affe28532d84097fef4a  -.HEIC
8c0832fa1ebf627c6e0714a4b6babcc1.HEIC
3b95f0841c21525f73d752193052fc9e.HEIC
609ae93438959bff10e7b96f8c24f779.HEIC
81b728654dded4e1be0b8d918d647cb3.HEIC
b287cb234bae7432b4f02b710fe97fcb.HEIC
eb8c546148633071f33261f3d329a2d6.HEIC
f003259eee0f7319b6e91a80c46b5974.HEIC
687f1d529d89358dab5f0c852cde3ca7.HEIC
d434fe19e84d0adc3b2dd9022ba0214d.HE